# Data Analysis Final Project
**Due Date:** January 29, 2026, 23:59  

**Group:** Cuircuit Synergy  

**Created By:** Jeremia Baumgartner, Lorenz Buchinger, Tim Zwölfer  

---
**Initial Setup**

In [1]:
# Initial setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plotting
plt.rcParams.update({
    'figure.figsize': [12, 8],
    'figure.dpi': 150,
    'figure.autolayout': True,
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'font.size': 12
})

sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.2)

np.random.seed(42)

---
**Load Data**

In [2]:
df_csv_data = pd.read_csv('traffic_accidents.csv')

---
## A. Data Preprocessing and Data Quality (70 points)
---
**Assigned to Lorenz**
- Dataset overview (dimensions, columns, types, time range, sampling rate, missingness
summary) (10 points)
- Basic statistical analysis using pandas (descriptives, grouped stats, quantiles) (10 points)
- Original data quality analysis with visualization (missingness patterns, outliers, dupli-
cates, timestamp gaps, inconsistent units) (20 points)
- Data preprocessing pipeline (cleaning steps, handling missing data, outliers strategy, re-
sampling or alignment if needed, feature engineering basics) (20 points)
- Preprocessed vs original comparison (before/after visuals plus commentary on what changed
and why) (10 points)

---
## B. Visualization and Exploratory Analysis (55 points)
---
**Assigned to Jeremia**
- Time-series visualizations (raw, smoothed, rolling mean or windowed views) (10 points)
- Distribution analysis with histograms and density style plots where applicable (10 points)
- Correlation analysis and heatmaps (Pearson and at least one alternative such as Spearman,
with short interpretation) (10 points)
- Daily or periodic pattern analysis (day-of-week, hour-of-day, seasonality indicators, or
test-cycle patterns) (15 points)
- Summary of observed patterns as short check statements (similar to True/False style)
with evidence (10 points)

---
## C. Probability and Event Analysis (45 points)
---
**Assigned to Tim**
- Threshold-based probability estimation for events (define event, justify threshold, compute
empirical probability) (15 points)
- Cross tabulation analysis for two variables (10 points)
- Conditional probability analysis (at least two meaningful conditional relationships) (15
points)
- Summary of observations and limitations (what could bias these estimates, what assump-
tions were made) (5 points)

---
## D. Statistical Theory Applications (45 points)
---
**Assigned to Tim**
- Law of Large Numbers demonstration (15 points)
- Central Limit Theorem application (sampling distributions, effect of sample size, interpretation) (25 points)
- Result interpretation and sanity checks (what would invalidate your conclusion, what you verified) (5 points)

---
## E. Regression and Predictive Modeling (45 points)
---
**Assigned to Lorenz**
- Define a prediction target and features (justify why they make sense) (10 points)
- Linear or polynomial model selection (include rationale and show at least two candidates)
(10 points)
- Model fitting and validation (train-test split appropriate for time-series. e.g., time-based split) (15 points)
- Residual analysis and interpretation (errors, bias, failure cases, what to improve next) (10 points)

---
## F. Dimensionality Reduction and Statistical Tests (40 points)
---
**Assigned to Jeremia**
### Part 1. Dimensionality Reduction (25 points)
- PCA projection and interpretation (variance explained, what clusters or separations mean) (10 points)
- t-SNE embedding with justified hyperparameters (perplexity or similar) and interpretation (7 points)
- UMAP embedding with justified hyperparameters (neighbors, min dist or similar) and interpretation (8 points)
### Part 2. Hypothesis Tests (15 points)
Perform at least three tests. Each test must include: null hypothesis, why the test is appropriate, assumptions, p-value, and practical interpretation.
- Chi-square test (choose one):
    - Chi-square test of independence (use a contingency table from two categorical or binned variables), or
    - Chi-square goodness-of-fit (compare observed counts to an expected distribution you justify). (5 points)
- One mean or location comparison test (choose one): t-test, Welch t-test, Mann-Whitney U, or ANOVA (5 points)
- One time-series relevant test (choose one): stationarity test (ADF or KPSS), Ljung-Box for autocorrelation, or change-point style test if justified (5 points)